# Introduction to Pyresample

https://pyresample.readthedocs.io/en/latest/

This package seems a bit more speciallized, and does not have as tight of integration with xarray like xESMF and Verde do. If working with satellite or swath data, this is not one to miss! This package integrates with Satpy https://satpy.readthedocs.io/en/stable/ .

#### (more) Integration with xarray

This is on the to-do list: https://pyresample.readthedocs.io/en/latest/roadmap.html?highlight=xarray#xarray-and-geoxarray

### Prerequisites

Knowing your way around pandas, xarray, numpy is beneficial. This is not deisgned to be an introduction to any of those packages. Would do this notebook after doing the xESMF one!

# Imports

In [1]:
import pandas as pd
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt

from appdirs import *

import dask.array as da

import pyresample
from pyresample import image, geometry
from pyresample.bilinear import NumpyBilinearResampler

from xarray import DataArray
from pyresample.bilinear import XArrayBilinearResampler

import os

In [2]:
%load_ext watermark
%watermark --iversions

sys       : 3.9.13 | packaged by conda-forge | (main, May 27 2022, 17:01:00) 
[Clang 13.0.1 ]
numpy     : 1.21.5
pandas    : 1.4.3
pyresample: 1.25.1
matplotlib: 3.5.2
xarray    : 0.20.1
dask      : 2022.7.0



# Loading in one netCDF

In [3]:
file = 'data/onestorm.nc'

Let's open this file with xarray:

In [4]:
ds = xr.open_dataset(file)
ds

<xarray.Dataset>
Dimensions:            (x: 768, y: 768, t: 12, x2: 192, y2: 192, x3: 384, y3: 384, x4: 48, y4: 48)
Dimensions without coordinates: x, y, t, x2, y2, x3, y3, x4, y4
Data variables:
    visible            (x, y, t) float32 ...
    water_vapor        (x2, y2, t) float32 ...
    clean_infrared     (x2, y2, t) float32 ...
    vil                (x3, y3, t) float32 ...
    lightning_flashes  (x4, y4, t) float32 ...
Attributes:
    t:        time dimension of all images. These are 5-min time steps
    x:        x pixel dimension of the visible imagery
    y:        y pixel dimension of the visible imagery
    x2:       x pixel dimension of the water vapor and infrared imagery
    y2:       y pixel dimension the water vapor and infrared imagery
    x3:       x pixel dimension of the vertically integrated liquid imagery
    y3:       y pixel dimension the vertically integrated liquid imagery
    x4:       x pixel dimension of the lightning flashes
    y4:       y pixel dimension of the lightning flashes

Trying to do the same thing with pyresample:

In [5]:
from pyresample.utils import load_cf_area

In [6]:
area_def, cf_info = load_cf_area('data/onestorm.nc', variable='visible', x='x', y='y')

ValueError: Found no AreaDefinition associated with variable visible (Variable x='x' is not a valid CF coordinate variable for the x axis)

This is supposed to fail. Will chat about pro's and con's in the summary.

# Resampling of gridded data using pyresample

Link to this turtorial is here: https://pyresample.readthedocs.io/en/latest/swath.html#pyresample-bilinear

We will be deconstructing it a bit to get into the details, but all of the code is from the above link.

In [7]:
target_def = geometry.AreaDefinition('areaD',
                                     'Europe (3km, HRV, VTC)',
                                     'areaD',
                                     {'a': '6378144.0', 'b': '6356759.0',
                                      'lat_0': '50.00', 'lat_ts': '50.00',
                                      'lon_0': '8.00', 'proj': 'stere'},
                                     800, 800,
                                     [-1370912.72, -909968.64,
                                      1029087.28, 1490031.36])

Unlike using xESMF, this does not depend or work with xarray:

In [8]:
print('target def type', type(target_def))

target def type <class 'pyresample.geometry.AreaDefinition'>


In [9]:
data = DataArray(da.from_array(np.fromfunction(lambda y, x: y*x, (500, 100))), dims=('y', 'x'))
type(data)

xarray.core.dataarray.DataArray

In [10]:
lons = da.from_array(np.fromfunction(lambda y, x: 3 + x * 0.1, (500, 100)))
lats = da.from_array(np.fromfunction(lambda y, x: 75 - y * 0.1, (500, 100)))
source_def = geometry.SwathDefinition(lons=lons, lats=lats)

In [11]:
type(lats)

dask.array.core.Array

In [12]:
resampler = XArrayBilinearResampler(source_def, target_def, 30e3)
result = resampler.resample(data)

/Users/tmartin/opt/anaconda3/envs/ml_scratch/lib/python3.9/site-packages/pyproj/crs/crs.py:1282: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)


## Can export to xarray

In [13]:
result.to_dataset()

<xarray.Dataset>
Dimensions:                                   (x: 800, y: 800)
Coordinates:
  * x                                         (x) float64 -1.369e+06 ... 1.02...
  * y                                         (y) float64 1.489e+06 ... -9.08...
Data variables:
    reshape-1df8237ed9770f6b6e8da20d988ab1c4  (y, x) float64 dask.array<chunksize=(800, 800), meta=np.ndarray>

In [14]:
data.to_dataset()

<xarray.Dataset>
Dimensions:                                 (y: 500, x: 100)
Dimensions without coordinates: y, x
Data variables:
    array-c6304ed7e79d5c61e4fb47cdb25246bf  (y, x) float64 dask.array<chunksize=(500, 100), meta=np.ndarray>

### Summary

Verde and xESMF are two different gridding packages, with two different aims. They both work well with xarray!